In [1]:
!python --version

Python 3.7.16


In [1]:
from Agent import Agent

import torch
import numpy as np

import os
os.chdir("..")
from rice import Rice

In [2]:
env = Rice(region_yamls_filename='fewer_region_yamls/')
initial_state = env.reset()

In [3]:
agents = [Agent(env, initial_state[agent_id]) for agent_id in range(env.num_agents)]

In [4]:
action = agents[0].proposalNet.select_action(torch.FloatTensor(initial_state[0]['features']))
action

array([1, 1, 1, 0, 1, 0, 1, 1, 0, 1, 0, 0, 0, 1, 0, 1, 1, 1, 0, 1, 0, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 1, 1, 1, 0, 0, 0, 0, 0, 1, 1, 0, 1,
       1, 0, 0, 1, 0, 1, 1, 0, 1, 0, 1, 1, 0, 0, 1, 1, 0, 1, 1, 1, 0, 1,
       1, 0, 1, 1, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 0, 1, 1,
       0, 1, 0, 1, 1, 1, 0, 0, 1, 1, 1, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0,
       0, 1, 0, 0, 0, 1, 1, 1, 0, 1, 1, 1, 0, 1, 1, 0, 1, 1, 0, 0, 0, 0,
       0, 1, 0, 0, 0, 1, 1, 1, 0, 1, 1, 0, 0, 0, 1, 1, 0, 1, 1, 0, 0, 0,
       1, 1, 1, 0, 0, 1, 1, 0, 0, 0, 1, 1, 1, 1, 1, 0, 1, 1, 1, 0, 0, 1,
       1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 1, 1, 0, 0,
       1, 0, 0, 0, 1, 1, 0, 1, 1, 1, 1, 1, 0, 0, 1, 1, 0, 1, 1, 1, 0, 1,
       0, 1, 0, 1, 0, 1, 1, 0, 1, 1, 0, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 1,
       0, 1, 0, 0, 0, 0, 1, 0, 0, 1, 0, 1, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0,
       0, 0, 0, 0, 1, 0, 0, 1, 0, 1, 1, 0, 0, 1, 0, 0, 0, 1, 1, 1, 0, 0,
       1, 0, 0, 0, 0, 1, 0, 1, 0, 1, 1, 0, 0, 0, 0,

In [5]:
f = torch.FloatTensor(initial_state[0]['features'])
prom = torch.FloatTensor(
    np.array([env.promises[agent_id] for agent_id in range(env.num_agents) if agent_id != 0]).flatten()
)
prop = torch.FloatTensor(
    np.array([env.proposals[sender_id][0] for sender_id in range(env.num_agents) if sender_id != 0]).flatten()
)

In [6]:
s = torch.cat((f, prom, prop))
s.shape

torch.Size([737])

In [7]:
action = agents[0].decisionNet.select_action(s)
action

array([0, 0, 1])